# Interacting with Campaigns <a class="anchor" id="top"></a>

In this notebook, you will deploy and interact with campaigns in Amazon Personalize.

1. [Introduction](#intro)
1. [Create campaigns](#create)
1. [Interact with campaigns](#interact)
1. [Batch recommendations](#batch)
1. [Wrap up](#wrapup)

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have several solutions and at least one solution version for each. Once a solution version is created, it is possible to get recommendations from them, and to get a feel for their overall behavior.

This notebook starts off by deploying each of the solution versions from the previous notebook into individual campaigns. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

As you with your customer on Amazon Personalize, you can modify the helper functions to fit the structure of their data input files to keep the additional rendering working.

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [1]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3
import pandas as pd

In [2]:
%store -r

## project constants

In [3]:
dataset_name = "movielense"
project_name = "sbcPersonalizePOC"

In [4]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Interact with campaigns <a class="anchor" id="interact"></a>
[Back to top](#top)

Now that all campaigns are deployed and active, we can start to get recommendations via an API call. Each of the campaigns is based on a different recipe, which behave in slightly different ways because they serve different use cases. We will cover each campaign in a different order than used in previous notebooks, in order to deal with the possible complexities in ascending order (i.e. simplest first).

First, let's create a supporting function to help make sense of the results returned by a Personalize campaign. Personalize returns only an `item_id`. This is great for keeping data compact, but it means you need to query a database or lookup table to get a human-readable result for the notebooks. We will create a helper function to return a human-readable result from the LastFM dataset.

Start by loading in the dataset which we can use for our lookup table.

In [5]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(dataset_dir + '/movies.csv', sep=',', usecols=[0,1], encoding='latin-1', dtype={'movieId': "object", 'title': "str"},index_col=0)

# Render some sample data
items_df.head(5)

,title
movieId,
1,Toy Story (1995)
2,Jumanji (1995)
3,Grumpier Old Men (1995)
4,Waiting to Exhale (1995)
5,Father of the Bride Part II (1995)


By defining the ID column as the index column it is trivial to return an artist by just querying the ID.

In [6]:
movie_id_example = 589
title = items_df.loc[movie_id_example]['title']
print(title)

Terminator 2: Judgment Day (1991)


That isn't terrible, but it would get messy to repeat this everywhere in our code, so the function below will clean that up.

In [7]:
def get_movie_by_id(movie_id, movie_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[int(movie_id)]['title']
    except:
        return "Error obtaining title"

Now let's test a few simple values to check our error catching.

In [8]:
# A known good id (The Princess Bride)
print(get_movie_by_id(movie_id="1197"))
# A bad type of value
print(get_movie_by_id(movie_id="987.9393939"))
# Really bad values
print(get_movie_by_id(movie_id="Steve"))

Princess Bride, The (1987)
Error obtaining title
Error obtaining title


Great! Now we have a way of rendering results. 

### SIMS

SIMS requires just an item as input, and it will return items which users interact with in similar ways to their interaction with the input item. In this particular case the item is a movie. 

The cells below will handle getting recommendations from SIMS and rendering the results. Let's see what the recommendations are for the first item we looked at earlier in this notebook (Terminator 2: Judgment Day).

In [10]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = sims_campaign_arn,
    itemId = str(589),
)

In [11]:
item_list = get_recommendations_response['itemList']
for item in item_list:
    print(get_movie_by_id(movie_id=item['itemId']))

Jurassic Park (1993)
Braveheart (1995)
Terminator, The (1984)
Fugitive, The (1993)
Speed (1994)
Crimson Tide (1995)
GoldenEye (1995)
Batman (1989)
Clear and Present Danger (1994)
True Lies (1994)
Mask, The (1994)
Die Hard: With a Vengeance (1995)
In the Line of Fire (1993)
Lion King, The (1994)
Ghost (1990)
Forrest Gump (1994)
Apollo 13 (1995)
Cliffhanger (1993)
Star Trek: Generations (1994)
Firm, The (1993)
Die Hard (1988)
Seven (a.k.a. Se7en) (1995)
Indiana Jones and the Last Crusade (1989)
Mission: Impossible (1996)
Mrs. Doubtfire (1993)


Congrats, this is your first list of recommendations! This list is fine, but it would be better to see the recommendations for our sample collection of artists render in a nice dataframe. Again, let's create a helper function to achieve this.

In [12]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the movie name
    movie_name = get_movie_by_id(movie_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = sims_campaign_arn,
        itemId = str(movie_ID),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

Now, let's test the helper function with several different movies. Let's sample some data from our dataset to test our SIMS campaign. Grab 5 random movies from our dataframe.

Note: We are going to show similar titles, so you may want to re-run the sample until you recognize some of the movies listed

In [13]:
samples = items_df.sample(5)
samples

,title
movieId,
2056,In Search of the Castaways (1962)
4583,Disorganized Crime (1989)
188675,Dogman (2018)
102378,Syrup (2013)
82765,Little Big Soldier (Da bing xiao jiang) (2010)


In [14]:
sims_recommendations_df = pd.DataFrame()
movies = samples.index.tolist()

for movie in movies:
    sims_recommendations_df = get_new_recommendations_df(sims_recommendations_df, movie)

sims_recommendations_df

,In Search of the Castaways (1962),Disorganized Crime (1989),Dogman (2018),Syrup (2013),Little Big Soldier (Da bing xiao jiang) (2010)
0,"Shawshank Redemption, The (1994)","Shawshank Redemption, The (1994)",The Man Who Killed Don Quixote (2018),"Shawshank Redemption, The (1994)","Shawshank Redemption, The (1994)"
1,Forrest Gump (1994),Forrest Gump (1994),How To Change The World (2015),Forrest Gump (1994),Forrest Gump (1994)
2,Pulp Fiction (1994),Pulp Fiction (1994),Tomorrow (2015),Pulp Fiction (1994),Pulp Fiction (1994)
3,"Silence of the Lambs, The (1991)","Silence of the Lambs, The (1991)",Shadow World (2016),"Silence of the Lambs, The (1991)","Silence of the Lambs, The (1991)"
4,Braveheart (1995),Braveheart (1995),The Salt of the Earth (2014),Braveheart (1995),Braveheart (1995)
5,"Matrix, The (1999)","Matrix, The (1999)",Citizenfour (2014),"Matrix, The (1999)","Matrix, The (1999)"
6,Schindler's List (1993),Schindler's List (1993),NaN,Schindler's List (1993),Schindler's List (1993)
7,Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode IV - A New Hope (1977),NaN,Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode IV - A New Hope (1977)
8,Jurassic Park (1993),Jurassic Park (1993),NaN,Jurassic Park (1993),Jurassic Park (1993)
9,Terminator 2: Judgment Day (1991),Terminator 2: Judgment Day (1991),NaN,Terminator 2: Judgment Day (1991),Terminator 2: Judgment Day (1991)


You may notice that a lot of the items look the same, hopefully not all of them do (this is more likely with a smaller # of interactions, which will be more common with the movielens small dataset). This shows that the evaluation metrics should not be the only thing you rely on when evaluating your solution version. So when this happens, what can you do to improve the results?

This is a good time to think about the hyperparameters of the Personalize recipes. The SIMS recipe has a `popularity_discount_factor` hyperparameter (see [documentation](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-sims.html)). Leveraging this hyperparameter allows you to control the nuance you see in the results. This parameter and its behavior will be unique to every dataset you encounter, and depends on the goals of the business. You can iterate on the value of this hyperparameter until you are satisfied with the results, or you can start by leveraging Personalize's hyperparameter optimization (HPO) feature. For more information on hyperparameters and HPO tuning, see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html).

### User Personalization

HRNN is one of the more advanced algorithms provided by Amazon Personalize. It supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

Since HRNN relies on having a sampling of users, let's load the data we need for that and select 3 random users. Since Movielens does not include user data, we will select 3 random numbers from the range of user id's in the dataset.

In [15]:
if not USE_FULL_MOVIELENS:
    users = random.sample(range(1, 600), 3)
else:
    users = random.sample(range(1, 162000), 3)
users

[578, 306, 431]

Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [16]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [17]:
recommendations_df_users = pd.DataFrame()
#users = users_df.sample(3).index.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

,578,306,431
0,"Devil Wears Prada, The (2006)",Sausage Party (2016),Jerry Maguire (1996)
1,"Notebook, The (2004)",The Emoji Movie (2017),Clerks (1994)
2,Yes Man (2008),Finding Dory (2016),Chasing Amy (1997)
3,How to Lose a Guy in 10 Days (2003),Despicable Me 2 (2013),Phenomenon (1996)
4,Love Actually (2003),Despicable Me (2010),Shakespeare in Love (1998)
5,Sherlock Holmes (2009),The Boss Baby (2017),Out of Sight (1998)
6,Meet the Fockers (2004),Rango (2011),So I Married an Axe Murderer (1993)
7,He's Just Not That Into You (2009),Minions (2015),Pretty in Pink (1986)
8,27 Dresses (2008),Wreck-It Ralph (2012),There's Something About Mary (1998)
9,Vicky Cristina Barcelona (2008),Ferdinand (2017),Clueless (1995)


Here we clearly see that the recommendations for each user are different. If you were to need a cache for these results, you could start by running the API calls through all your users and store the results, or you could use a batch export, which will be covered later in this notebook.

Now lets apply item filters to see recommendations for one of these users within a genre


In [18]:
def get_new_recommendations_df_by_filter(recommendations_df, user_id, filter_arn):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
        filterArn = filter_arn
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    filter_name = filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

You can see the recommendations for movies within a given genre. Within a VOD application you could create Shelves (also known as rails or carosels) easily by using these filters. Depending on the information you have about your items, You could also filter on additional information such as keyword, year/decade etc.

In [19]:
recommendations_df_shelves = pd.DataFrame()
for filter_arn in meta_filter_arns:
    recommendations_df_shelves = get_new_recommendations_df_by_filter(recommendations_df_shelves, user, filter_arn)
for filter_arn in decade_filter_arns:
    recommendations_df_shelves = get_new_recommendations_df_by_filter(recommendations_df_shelves, user, filter_arn)

recommendations_df_shelves

,IMAX,Drama,Documentary,Mystery,Film-Noir,Horror,Fantasy,1970s
0,Apollo 13 (1995),Jerry Maguire (1996),Hoop Dreams (1994),Conspiracy Theory (1997),L.A. Confidential (1997),"Blair Witch Project, The (1999)",Being John Malkovich (1999),Manhattan (1979)
1,Beauty and the Beast (1991),Chasing Amy (1997),From the Earth to the Moon (1998),Basic Instinct (1992),Dark City (1998),Arachnophobia (1990),Big (1988),Saturday Night Fever (1977)
2,"Lion King, The (1994)",Phenomenon (1996),Roger & Me (1989),"Godfather: Part III, The (1990)","Grifters, The (1990)",Scream (1996),Pleasantville (1998),Grease (1978)
3,Star Wars: Episode II - Attack of the Clones (...,Shakespeare in Love (1998),Everest (1998),Masquerade (1988),Devil in a Blue Dress (1995),Surf Nazis Must Die (1987),Jumanji (1995),Rocky (1976)
4,Spider-Man 2 (2004),Out of Sight (1998),T-Rex: Back to the Cretaceous (1998),"Game, The (1997)",2 Days in the Valley (1996),Dracula (Bram Stoker's Dracula) (1992),Groundhog Day (1993),"Goodbye Girl, The (1977)"
5,Poseidon (2006),Pretty in Pink (1986),For All Mankind (1989),The Devil's Advocate (1997),Mulholland Drive (2001),Scream 3 (2000),"Honey, I Shrunk the Kids (1989)",Deliverance (1972)
6,"Matrix Reloaded, The (2003)",Titanic (1997),Dancemaker (1998),L.A. Confidential (1997),Bitter Moon (1992),"Fly, The (1986)",Ghost (1990),The Star Wars Holiday Special (1978)
7,More (1998),"American President, The (1995)",Looking for Richard (1996),"Talented Mr. Ripley, The (1999)",Down by Law (1986),Scream 2 (1997),"Crow, The (1994)",M*A*S*H (a.k.a. MASH) (1970)
8,Everest (1998),Conspiracy Theory (1997),"Big One, The (1997)",Mission: Impossible (1996),Miller's Crossing (1990),"Mummy, The (1999)",Like Water for Chocolate (Como agua para choco...,Harold and Maude (1971)
9,"Matrix Revolutions, The (2003)",Rushmore (1998),Pirates of Silicon Valley (1999),Eyes Wide Shut (1999),Lost Highway (1997),Wolf (1994),"Indian in the Cupboard, The (1995)",Swept Away (Travolti da un insolito destino ne...


The next topic is real-time events. Personalize has the ability to listen to events from your application in order to update the recommendations shown to the user. This is especially useful in media workloads, like video-on-demand, where a customer's intent may differ based on if they are watching with their children or on their own.

Additionally the events that are recorded via this system are stored until a delete call from you is issued, and they are used as historical data alongside the other interaction data you provided when you train your next models.

#### Real time events

Start by creating an event tracker that is attached to the campaign.

In [20]:
event_tracker_name = f"{project_name}_MovieTracker"

response = personalize.create_event_tracker(
    name='MovieTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

arn:aws:personalize:us-east-1:726011567823:event-tracker/87c96729
391f3133-33f1-42f6-92b3-48f26e7725e1


We will create some code that simulates a user interacting with a particular item. After running this code, you will get recommendations that differ from the results above.

We start by creating some methods for the simulation of real time events.

In [21]:
session_dict = {}

def send_movie_click(USER_ID, ITEM_ID, EVENT_TYPE):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= str(USER_ID),
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': str(EVENT_TYPE),
        'properties': event_json
        }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id, event_type):
    # Get the artist name (header of column)
    movie_name = get_movie_by_id(item_id)
    # Interact with different movies
    print('sending event ' + event_type + ' for ' + get_movie_by_id(item_id))
    send_movie_click(USER_ID=user_id, ITEM_ID=item_id, EVENT_TYPE=event_type)
    # Get the recommendations (note you should have a base recommendation DF created before)
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        artist = get_movie_by_id(item['itemId'])
        recommendation_list.append(artist)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    #recommendations_df = recommendations_df.join(new_rec_DF)
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

At this point, we haven't generated any real-time events yet; we have only set up the code. To compare the recommendations before and after the real-time events, let's pick one user and generate the original recommendations for them.

In [22]:
# First pick a user
user_id = user

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    artist = get_movie_by_id(item['itemId'])
    recommendation_list.append(artist)
user_recommendations_df = pd.DataFrame(recommendation_list, columns = [user_id])
user_recommendations_df

,431
0,Jerry Maguire (1996)
1,Clerks (1994)
2,Chasing Amy (1997)
3,Phenomenon (1996)
4,Shakespeare in Love (1998)
5,Out of Sight (1998)
6,So I Married an Axe Murderer (1993)
7,Pretty in Pink (1986)
8,There's Something About Mary (1998)
9,Clueless (1995)


Ok, so now we have a list of recommendations for this user before we have applied any real-time events. Now let's pick 3 random artists which we will simulate our user interacting with, and then see how this changes the recommendations.

In [23]:
# Next generate 3 random movies
movies = items_df.sample(3).index.tolist()

In [24]:
# Note this will take about 15 seconds to complete due to the sleeps
for movie in movies:
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, user_id, movie,'click')
    time.sleep(5)

sending event click for Apple Dumpling Gang, The (1975)
sending event click for The Lobster (2015)
sending event click for Sophie Scholl: The Final Days (Sophie Scholl - Die letzten Tage) (2005)


Now we can look at how the click events changed the recommendations.

In [25]:
user_recommendations_df

,431,"Apple Dumpling Gang, The (1975)",The Lobster (2015),Sophie Scholl: The Final Days (Sophie Scholl - Die letzten Tage) (2005)
0,Jerry Maguire (1996),Jerry Maguire (1996),Blazing Saddles (1974),Sixteen Candles (1984)
1,Clerks (1994),Clerks (1994),Grease (1978),Fast Times at Ridgemont High (1982)
2,Chasing Amy (1997),Chasing Amy (1997),Stripes (1981),Grease (1978)
3,Phenomenon (1996),Phenomenon (1996),Sixteen Candles (1984),Pretty in Pink (1986)
4,Shakespeare in Love (1998),Shakespeare in Love (1998),Caddyshack (1980),Blazing Saddles (1974)
5,Out of Sight (1998),Out of Sight (1998),"Blues Brothers, The (1980)",Manhattan (1979)
6,So I Married an Axe Murderer (1993),So I Married an Axe Murderer (1993),"Christmas Story, A (1983)",Tootsie (1982)
7,Pretty in Pink (1986),Pretty in Pink (1986),Romancing the Stone (1984),When Harry Met Sally... (1989)
8,There's Something About Mary (1998),There's Something About Mary (1998),Pretty in Pink (1986),Back to the Future Part III (1990)
9,Clueless (1995),Clueless (1995),Manhattan (1979),Risky Business (1983)


In the cell above, the first column after the index is the user's default recommendations from User Personalization, and each column after that has a header of the artist that they interacted with via a real time event, and the recommendations after this event occurred. 

The behavior may not shift very much; this is due to the relatively limited nature of this dataset and effect of a few random clicks. If you wanted to better understand this, try simulating clicking more movies, and you should see a more pronounced impact.

Now lets look at the event filters, which allow you to filter items based on the interaction data. For this dataset, it could be click or watch based on the data we imported, but could be based on whatever interaction schema you design (click, rate, like, watch, purchase etc.) For VOD shelves you could move a title from "Top picks for you" to a "Watch again", the watch again recommendations will be based on the users current interactions, but only recommend titles that have already been watched.


In [43]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

,watched,unwatched
0,"Boot, Das (Boat, The) (1981)",Hotel Rwanda (2004)
1,"Sting, The (1973)",Master and Commander: The Far Side of the Worl...
2,True Romance (1993),"Last Samurai, The (2003)"
3,So I Married an Axe Murderer (1993),Troy (2004)
4,Young Frankenstein (1974),The Alamo (2004)
5,"Game, The (1997)",Open Range (2003)
6,NaN,Flags of Our Fathers (2006)
7,NaN,Australia (2008)
8,NaN,Lord of War (2005)
9,NaN,Letters from Iwo Jima (2006)


now lets send a watch event in for 4 unwatched recommendations, which would simulate watching 4 movies. In a VOD application, you may choose to send in an event after they have watched a significant amount (over 75%) of a piece of content. Sending at 100% complete could miss people that stop short of the credits.

In [44]:
 # Get the recommendations
top_unwatched_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = userpersonalization_campaign_arn,
    userId = str(user_id),
    filterArn = filter_arn,
    numResults=4)
item_list = top_unwatched_recommendations_response['itemList']
for item in item_list:
    print('sending event watch for ' + get_movie_by_id(item['itemId']))
    send_movie_click(USER_ID=user_id, ITEM_ID=item['itemId'], EVENT_TYPE='watch')
    time.sleep(10)

sending event watch for Hotel Rwanda (2004)
sending event watch for Bandidas (2006)
sending event watch for Master and Commander: The Far Side of the World (2003)
sending event watch for Last Samurai, The (2003)


Now we can look at the event filters to see the updated watched and unwatched recommendations 

In [45]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

,watched,unwatched
0,Hotel Rwanda (2004),Sin City (2005)
1,"Last Samurai, The (2003)",Lord of War (2005)
2,Bandidas (2006),Cars (2006)
3,Master and Commander: The Far Side of the Worl...,300 (2007)
4,"Boot, Das (Boat, The) (1981)",Batman Begins (2005)
5,"Game, The (1997)",Casino Royale (2006)
6,"Sting, The (1973)",Ocean's Twelve (2004)
7,True Romance (1993),"40-Year-Old Virgin, The (2005)"
8,Young Frankenstein (1974),"Incredibles, The (2004)"
9,So I Married an Axe Murderer (1993),Hot Fuzz (2007)


### Personalized Ranking

The core use case for personalized ranking is to take a collection of items and to render them in priority or probable order of interest for a user. For a VOD application you want dynamically render a personalized shelf/rail/carousel based on some information (director, location, superhero franchise, movie time period etc). This may not be information that you have in your metadata, so a item metadata filter will not work, howeverr you may have this information within you system to generate the item list. 

To demonstrate this, we will use the same user from before and a random collection of items.

In [46]:
rerank_user = user
rerank_items = items_df.sample(25).index.tolist()

Now build a nice dataframe that shows the input data.

In [47]:
rerank_list = []
for item in rerank_items:
    movie = get_movie_by_id(item)
    rerank_list.append(movie)
rerank_df = pd.DataFrame(rerank_list, columns = ['Un-Ranked'])
rerank_df

,Un-Ranked
0,Josie and the Pussycats (2001)
1,High Spirits (1988)
2,"Iron Giant, The (1999)"
3,Vanilla Sky (2001)
4,Reptilicus (1961)
5,"Nightmare Before Christmas, The (1993)"
6,Mallrats (1995)
7,Battleship Potemkin (1925)
8,"Children, The (2008)"
9,"Wedding Planner, The (2001)"


Then make the personalized ranking API call.

In [48]:
# Convert user to string:
user_id = str(rerank_user)
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

Now add the reranked items as a second column to the original dataframe, for a side-by-side comparison.

In [49]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    movie = get_movie_by_id(item['itemId'])
    ranked_list.append(movie)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

,Un-Ranked,Re-Ranked
0,Josie and the Pussycats (2001),For a Few Dollars More (Per qualche dollaro in...
1,High Spirits (1988),Sneakers (1992)
2,"Iron Giant, The (1999)",Battleship Potemkin (1925)
3,Vanilla Sky (2001),School for Scoundrels (2006)
4,Reptilicus (1961),Faster Pussycat! Kill! Kill! (1965)
5,"Nightmare Before Christmas, The (1993)","Nightmare Before Christmas, The (1993)"
6,Mallrats (1995),Mallrats (1995)
7,Battleship Potemkin (1925),Babes in Toyland (1961)
8,"Children, The (2008)",Kung Fu Panda: Secrets of the Furious Five (2008)
9,"Wedding Planner, The (2001)",My Life in Pink (Ma vie en rose) (1997)


You can see above how each entry was re-ordered based on the model's understanding of the user. This is a popular task when you have a collection of items to surface a user, a list of promotions for example.

## Batch recommendations <a class="anchor" id="batch"></a>
[Back to top](#top)

There are many cases where you may want to have a larger dataset of exported recommendations. Recently, Amazon Personalize launched batch recommendations as a way to export a collection of recommendations to S3. In this example, we will walk through how to do this for the HRNN solution. For more information about batch recommendations, please see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/getting-recommendations.html#recommendations-batch). This feature applies to all recipes, but the output format will vary.

A simple implementation looks like this:

```python
import boto3

personalize_rec = boto3.client(service_name='personalize')

personalize_rec.create_batch_inference_job (
    solutionVersionArn = "Solution version ARN",
    jobName = "Batch job name",
    roleArn = "IAM role ARN",
    jobInput = 
       {"s3DataSource": {"path": S3 input path}},
    jobOutput = 
       {"s3DataDestination": {"path":S3 output path"}}
)
```

The SDK import, the solution version arn, and role arns have all been determined. This just leaves an input, an output, and a job name to be defined.

Starting with the input for HRNN, it looks like:


```JSON
{"userId": "4638"}
{"userId": "663"}
{"userId": "3384"}
```

This should yield an output that looks like this:

```JSON
{"input":{"userId":"4638"}, "output": {"recommendedItems": ["296", "1", "260", "318"]}}
{"input":{"userId":"663"}, "output": {"recommendedItems": ["1393", "3793", "2701", "3826"]}}
{"input":{"userId":"3384"}, "output": {"recommendedItems": ["8368", "5989", "40815", "48780"]}}
```

The output is a JSON Lines file. It consists of individual JSON objects, one per line. So we will need to put in more work later to digest the results in this format.

### Building the input file

When you are using the batch feature, you specify the users that you'd like to receive recommendations for when the job has completed. The cell below will again select a few random users and will then build the file and save it to disk. From there, you will upload it to S3 to use in the API call later.

In [50]:
# We will use the same users from before
users
# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir + "/" + json_input_filename, 'w') as json_input:
    for user_id in users:
        json_input.write('{"userId": "' + str(user_id) + '"}\n')

In [51]:
# Showcase the input file:
!cat $data_dir"/"$json_input_filename

{"userId": "578"}
{"userId": "306"}
{"userId": "431"}


Upload the file to S3 and save the path as a variable for later.

In [52]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(data_dir+"/"+json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

s3://726011567823-us-east-1-sbcpersonalizepocvod-sbc/json_input.json


Batch recommendations read the input from the file we've uploaded to S3. Similarly, batch recommendations will save the output to file in S3. So we define the output path where the results should be saved.

In [53]:
# Define the output path
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

s3://726011567823-us-east-1-sbcpersonalizepocvod-sbc/


Now just make the call to kick off the batch export process.

In [54]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = userpersonalization_solution_version_arn,
    jobName = "VOD-POC-Batch-Inference-Job-UserPersonalization_" + str(round(time.time()*1000)),
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

Run the while loop below to track the status of the batch recommendation call. This can take around 30 minutes to complete, because Personalize needs to stand up the infrastructure to perform the task. We are testing the feature with a dataset of only 3 users, which is not an efficient use of this mechanism. Normally, you would only use this feature for bulk processing, in which case the efficiencies will become clear.

In [ ]:
current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Import Started on:  05:38:41 PM
DatasetInferenceJob: CREATE PENDING


In [ ]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, data_dir+"/"+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(data_dir+"/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            movie = get_movie_by_id(item)
            recommendation_list.append(movie)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

Use these notebooks as a guide to getting started with your customers for POCs. As you find missing components, or discover new approaches, cut a pull request and provide any additional helpful components that may be missing from this collection.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in `06_Clean_Up_Resources.ipynb`.

In [ ]:
import os
from utils import save_json

os.makedirs("results", exist_ok=True)

data = {
"event_tracker_arn": event_tracker_arn,
"batchInferenceJobArn": batchInferenceJobArn
}

save_json("results/05.json", data=data)

In [ ]:
%store event_tracker_arn
%store batchInferenceJobArn